In [ ]:

import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
from google.colab import drive
drive.mount('/content/drive')
# set project path to "dallas airbnb" folder
project_path = "/content/drive/My Drive/springboard/dallas airbnb"

# change into that folder
os.chdir(project_path)
print("Now in:", project_path)
print("Files here:", os.listdir(project_path))

# load listings.csv if it exists
if "listings.csv" in os.listdir(project_path):
    listings = pd.read_csv("listings.csv")
    print("✅ Listings loaded:", listings.shape)
    display(listings.head())
else:
    print("⚠️ listings.csv not found in", project_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Now in: /content/drive/My Drive/springboard/dallas airbnb
Files here: ['listings.csv', 'neighbourhoods.geojson', 'models', 'Dallas_AIRBNB.ipynb', 'Untitled0.ipynb', '11.5 Dallas_AIRBNB.ipynb']
✅ Listings loaded: (5531, 18)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,61878,MODERN LIVING AND FURNISHINGS,300211,Rita,NaN,District 2,32.816900,-96.820180,Entire home/apt,111.0,30,57,2025-04-30,0.32,1,288,2,NaN
1,795703,Amazing location walk to Downtown Dallas,4191322,Michelle,NaN,District 14,32.803270,-96.809760,Entire home/apt,243.0,30,71,2024-07-27,0.47,1,269,1,NaN
2,860248,"Peaceful Home By the Lake: Safe, Cozy, Quiet",4505460,Judy,NaN,District 9,32.855090,-96.706250,Private room,44.0,7,72,2023-09-30,0.67,2,151,0,NaN
3,1154424,"Uptown, Charming Studio 1B,Fast Free WiFi Parking",6063232,Joan,NaN,District 14,32.818684,-96.790154,Entire home/apt,92.0,3,97,2024-10-29,0.66,9,310,3,NaN
4,1277933,The Santa Fe Suite and Maxfield Parrish Suite,6904484,C. F. Sandy,NaN,District 11,32.924490,-96.790620,Private room,55.0,1,30,2019-11-09,0.30,1,166,0,NaN


In [ ]:
# drop unnecessary columns
listings_clean = listings.drop(
    columns=["name", "host_id", "host_name", "neighbourhood_group", "last_review", "license"],
    errors="ignore"  # ignore avoids errors if column missing
)

print("Remaining columns:", listings_clean.columns.tolist())
print("Shape:", listings_clean.shape)
listings_clean.head()


Remaining columns: ['id', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'number_of_reviews_ltm']
Shape: (5531, 12)


,id,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,61878,District 2,32.816900,-96.820180,Entire home/apt,111.0,30,57,0.32,1,288,2
1,795703,District 14,32.803270,-96.809760,Entire home/apt,243.0,30,71,0.47,1,269,1
2,860248,District 9,32.855090,-96.706250,Private room,44.0,7,72,0.67,2,151,0
3,1154424,District 14,32.818684,-96.790154,Entire home/apt,92.0,3,97,0.66,9,310,3
4,1277933,District 11,32.924490,-96.790620,Private room,55.0,1,30,0.30,1,166,0


In [ ]:
# BLOCK 3 — Cleaning, type fixes, and initial encoding

# Make a working copy
df = listings_clean.copy()

# ---- Fill categorical columns ----
if "neighbourhood" in df.columns:
    df["neighbourhood"] = df["neighbourhood"].fillna("Unknown")
if "room_type" in df.columns:
    df["room_type"] = df["room_type"].fillna("Unknown")

# ---- Remove "Hotel room" entries ----
if "room_type" in df.columns:
    before = len(df)
    df = df[df["room_type"] != "Hotel room"]
    after = len(df)
    print(f"Removed {before - after} 'Hotel room' rows")
    print(f"Remaining rows: {after}")

# ---- Convert room_type to numeric codes and drop original ----
if "room_type" in df.columns:
    room_type_map = {
        "Entire home/apt": 1,
        "Private room": 2,
        "Shared room": 3
    }
    df["room_type_numeric"] = (
        df["room_type"]
        .map(room_type_map)
        .fillna(0)
        .astype(int)
    )
    df = df.drop(columns=["room_type"], errors="ignore")

# ---- Convert likely numeric columns to numeric dtype ----
likely_numeric = [
    "price", "minimum_nights", "availability_365",
    "number_of_reviews", "number_of_reviews_ltm", "reviews_per_month",
    "calculated_host_listings_count", "latitude", "longitude"
]

for col in likely_numeric:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# ---- Fill numeric columns (0 or median) ----
fill_zero = [
    c for c in [
        "availability_365",
        "number_of_reviews",
        "number_of_reviews_ltm",
        "reviews_per_month"
    ] if c in df.columns
]

if fill_zero:
    df[fill_zero] = df[fill_zero].fillna(0)

fill_median = [
    c for c in [
        "minimum_nights",
        "calculated_host_listings_count",
        "price"
    ] if c in df.columns
]

for c in fill_median:
    if c in df.columns:
        df[c] = df[c].fillna(df[c].median())

# ---- Drop rows missing essential geo fields ----
must_have = [c for c in ["latitude", "longitude"] if c in df.columns]
if must_have:
    df = df.dropna(subset=must_have)

# ---- Encode neighbourhood as numeric ----
if "neighbourhood" in df.columns:
    df["neighbourhood_encoded"] = (
        df["neighbourhood"].astype("category").cat.codes
    )

# ---- Quick data quality report ----
print("\nMissing values per column:")
print(df.isnull().sum().sort_values(ascending=False))

print("\nDuplicate rows:", df.duplicated().sum())
if "id" in df.columns:
    print("Duplicate IDs:", df["id"].duplicated().sum())

for col in ["price", "minimum_nights", "availability_365", "reviews_per_month"]:
    if col in df.columns:
        print(
            f"\n{col}: min={df[col].min()}, max={df[col].max()}, "
            f"mean={df[col].mean():.2f}, median={df[col].median():.2f}"
        )

df.head()


Removed 47 'Hotel room' rows
Remaining rows: 5484

Missing values per column:
id                                0
neighbourhood                     0
latitude                          0
longitude                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
number_of_reviews_ltm             0
room_type_numeric                 0
neighbourhood_encoded             0
dtype: int64

Duplicate rows: 0
Duplicate IDs: 0

price: min=9.0, max=50000.0, mean=305.73, median=124.00

minimum_nights: min=1, max=365, mean=8.18, median=2.00

availability_365: min=0, max=365, mean=249.54, median=298.00

reviews_per_month: min=0.0, max=25.27, mean=1.74, median=1.03


,id,neighbourhood,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,room_type_numeric,neighbourhood_encoded
0,61878,District 2,32.816900,-96.820180,111.0,30,57,0.32,1,288,2,1,6
1,795703,District 14,32.803270,-96.809760,243.0,30,71,0.47,1,269,1,1,5
2,860248,District 9,32.855090,-96.706250,44.0,7,72,0.67,2,151,0,2,13
3,1154424,District 14,32.818684,-96.790154,92.0,3,97,0.66,9,310,3,1,5
4,1277933,District 11,32.924490,-96.790620,55.0,1,30,0.30,1,166,0,2,2


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Fix latitude if some values start with a quote character
if "latitude" in df.columns:
    df["latitude"] = df["latitude"].astype(str).str.lstrip("'").astype(float)

# --- Filters ---
before = len(df)

df = df[(df["price"] > 0) & (df["price"] <= 1000)]
df = df[(df["minimum_nights"] >= 1) & (df["minimum_nights"] <= 365)]
df = df[df["availability_365"].between(0, 365)]
df = df[df["reviews_per_month"] >= 0]

after = len(df)
print(f"Rows before filtering: {before}")
print(f"Rows after filtering:  {after}")
print(f"Dropped rows: {before - after}")

# --- Create binary column: 1 if Entire home/apt, else 0 ---
if "is_entire_home" not in df.columns:
    if "room_type_numeric" in df.columns:
        # 1 = Entire home/apt (your mapping), 2 = Private room, 3 = Shared room
        df["is_entire_home"] = (df["room_type_numeric"] == 1).astype(int)
    elif "room_type" in df.columns:
        df["is_entire_home"] = (df["room_type"] == "Entire home/apt").astype(int)
    else:
        raise KeyError("Neither 'room_type_numeric' nor 'room_type' is present to create 'is_entire_home'.")

# --- Encode neighbourhood as numeric ---
encoder = LabelEncoder()
df["neighbourhood_encoded"] = encoder.fit_transform(df["neighbourhood"])

# Quick check
print(df[["neighbourhood", "neighbourhood_encoded"]].head(10))
print(df.head())


Rows before filtering: 5484
Rows after filtering:  5392
Dropped rows: 92
  neighbourhood  neighbourhood_encoded
0    District 2                      6
1   District 14                      5
2    District 9                     13
3   District 14                      5
4   District 11                      2
5   District 14                      5
6   District 14                      5
7   District 14                      5
8    District 2                      6
9   District 14                      5
        id neighbourhood   latitude  longitude  price  minimum_nights  \
0    61878    District 2  32.816900 -96.820180  111.0              30   
1   795703   District 14  32.803270 -96.809760  243.0              30   
2   860248    District 9  32.855090 -96.706250   44.0               7   
3  1154424   District 14  32.818684 -96.790154   92.0               3   
4  1277933   District 11  32.924490 -96.790620   55.0               1   

   number_of_reviews  reviews_per_month  calculated_host_li

In [ ]:
# BLOCK 5 — Feature Engineering

# 1. Price weighted by room type
df["price_weighted_roomtype"] = df["price"] * df["room_type_numeric"]

# 2. Potential revenue (annual)
# availability_365 = how many days NOT booked
# booked_days = 365 - availability
df["potential_revenue"] = df["price"] * (365 - df["availability_365"])

print(df[["price", "room_type_numeric", "price_weighted_roomtype",
          "availability_365", "potential_revenue"]].head())


   price  room_type_numeric  price_weighted_roomtype  availability_365  \
0  111.0                  1                    111.0               288   
1  243.0                  1                    243.0               269   
2   44.0                  2                     88.0               151   
3   92.0                  1                     92.0               310   
4   55.0                  2                    110.0               166   

   potential_revenue  
0             8547.0  
1            23328.0  
2             9416.0  
3             5060.0  
4            10945.0  


In [27]:
# BLOCK 6 — Feature Definition and Data Split (CRITICAL FIX)

target_col = "potential_revenue"

# Define the final set of features needed for the Pipeline.
# We explicitly list them to ensure the ColumnTransformer finds them all.
X_features = [
    'price', 'minimum_nights', 'number_of_reviews', 'calculated_host_listings_count',
    'availability_365', 'number_of_reviews_ltm', 'reviews_per_month',
    'room_type_numeric', 'price_weighted_roomtype', 'is_entire_home',
    'latitude', 'longitude',
    'neighbourhood' # CRITICAL: This categorical column must be kept for the Pipeline (Block 7)
]

# Drop redundant or non-feature columns from df, then select the features
X = df.drop(columns=[target_col, "id", "neighbourhood_encoded"], errors='ignore')
X = X[X_features]
y = df[target_col]

# Split data into training and testing sets (80% train, 20% test)
# Data is kept raw (not scaled/encoded) here.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Features in X (Raw data before preprocessing): {X.columns.tolist()}")



X_train shape: (4313, 13)
X_test shape: (1079, 13)
Features in X (Raw data before preprocessing): ['price', 'minimum_nights', 'number_of_reviews', 'calculated_host_listings_count', 'availability_365', 'number_of_reviews_ltm', 'reviews_per_month', 'room_type_numeric', 'price_weighted_roomtype', 'is_entire_home', 'latitude', 'longitude', 'neighbourhood']


In [28]:
# BLOCK 7 — Preprocessing Pipeline Definition (ColumnTransformer)

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# 1. Define feature sets

numerical_features = [
    'price', 'minimum_nights', 'number_of_reviews',
    'calculated_host_listings_count', 'availability_365',
    'number_of_reviews_ltm', 'reviews_per_month',
    'room_type_numeric', 'price_weighted_roomtype',
    'is_entire_home', 'latitude', 'longitude'
]
# Categorical features: These require One-Hot Encoding
categorical_features = ['neighbourhood']

# 2. Create Preprocessor:
# Scaling for numerical features, One-Hot for categorical
preprocessor = ColumnTransformer(
    transformers=[
        # 1. Apply One-Hot Encoding to categorical (nominal) features
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features),
        # 2. Apply Standard Scaling to numerical features
        ('num_scale', StandardScaler(), numerical_features)
    ],
    # Drop any remaining columns (e.g., id)
    remainder='drop'
)

print("Preprocessing Pipeline (ColumnTransformer) defined.")

Preprocessing Pipeline (ColumnTransformer) defined.


In [29]:
# BLOCK 8 — Model Training and Comparison with Pipelines

from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor # We should include XGBoost since you used it.

# Define the evaluation function
def evaluate_regression(y_true, y_pred, label="Model"):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

# Define Models (Using hyperparameters from your executed notebook)
models = {
    'Ridge Regression': Ridge(alpha=1.0, random_state=42),
    'Random Forest Regressor': RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1),
    # Using XGBoost as in your notebook, not generic Gradient Boosting
    'XGBoost Regressor': XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, random_state=42, n_jobs=-1)
}

results = {}

print("\nStarting Model Training and Evaluation...")

for name, model in models.items():
    # 1. Create a full Pipeline (Preprocessor + Model)
    full_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                    ('regressor', model)])

    # 2. Train the model (Preprocessor FITS ONLY on X_train)
    full_pipeline.fit(X_train, y_train)

    # 3. Predict and evaluate on TEST set
    y_test_pred = full_pipeline.predict(X_test)
    test_mae, test_rmse, test_r2 = evaluate_regression(y_test, y_test_pred, label=name)

    # 4. Predict and evaluate on TRAIN set (for overfitting check)
    y_train_pred = full_pipeline.predict(X_train)
    train_mae, train_rmse, train_r2 = evaluate_regression(y_train, y_train_pred, label=f"{name} (Train)")

    results[name] = {
        'Train_R2': train_r2,
        'Test_R2': test_r2,
        'Test_MAE': test_mae,
        'Test_RMSE': test_rmse
    }

    print(f"✅ {name} complete. Test R2: {test_r2:.4f}")

# Display Results
results_df = pd.DataFrame(results).T
print("\n--- Final Model Comparison Results (TEST SET) ---")
print(results_df.sort_values(by='Test_R2', ascending=False))


Starting Model Training and Evaluation...
✅ Ridge Regression complete. Test R2: 0.7189
✅ Random Forest Regressor complete. Test R2: 0.9931
✅ XGBoost Regressor complete. Test R2: 0.9938

--- Final Model Comparison Results (TEST SET) ---
                         Train_R2   Test_R2     Test_MAE     Test_RMSE
XGBoost Regressor        0.999947  0.993787   522.406501   2089.882415
Random Forest Regressor  0.998996  0.993081   567.905653   2205.343967
Ridge Regression         0.723037  0.718863  7528.921288  14057.784468
